# Serving

In [ ]:
!pip3 install -Uq grpcio==1.26.0
!pip3 install -q matplotlib
!pip3 install -q tensorflow-datasets

In [ ]:
import os
import subprocess
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

**as_supervised**: Returns tuple `(img, label)` instead of dict `{'image': img, 'label': label}`

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
assert isinstance(ds_train, tf.data.Dataset)
assert isinstance(ds_test, tf.data.Dataset)

In [ ]:
tfds.visualization.show_examples(ds_train, ds_info)

In [ ]:
#tfds.core.benchmark(train_ds, batch_size=32)

In [ ]:
@tf.function
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

In [ ]:
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
tfds.core.benchmark(ds_train, batch_size=32)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax, name='Softmax')
])
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.summary()

In [ ]:
model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_test,
)